# Обучение с учителем для решения задач коммивояжера


In [8]:
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Скрывает INFO-логи (оставляет WARNING и ERROR)
tf.get_logger().setLevel('ERROR')  # Скрывает большинство логов TensorFlow
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

2025-04-15 05:59:31.598935: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744696771.658939    1407 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744696771.673498    1407 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744696771.773146    1407 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744696771.773161    1407 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744696771.773163    1407 computation_placer.cc:177] computation placer alr

In [9]:
import numpy as np
from tensorflow.keras.layers import Input, Dense, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tqdm import tqdm
from python_tsp.exact import solve_tsp_dynamic_programming, solve_tsp_branch_and_bound
from python_tsp.heuristics import solve_tsp_local_search, solve_tsp_simulated_annealing
from python_tsp.heuristics import solve_tsp_lin_kernighan, solve_tsp_record_to_record
from sklearn.metrics import mean_absolute_percentage_error

### Модель предсказывает какие дуги входят в маршрут, но не их последовательность

In [10]:
class TSPSolver:
    def __init__(self, num_cities, hidden_dim=256):
        self.num_cities = num_cities
        self.hidden_dim = hidden_dim
        self.model = self._build_model()
    
    def _build_model(self):
        # Вход: матрица расстояний (batch, cities, cities)
        inputs = Input(shape=(self.num_cities, self.num_cities))
        
        # Кодировщик на основе полносвязных слоев
        x = Dense(self.hidden_dim, activation='relu')(inputs)
        x = LayerNormalization()(x)
        x = Dense(self.hidden_dim, activation='relu')(x)
        x = LayerNormalization()(x)

        x = Dense(self.hidden_dim, activation='relu')(x)
        x = LayerNormalization()(x)
        
        # Выходной слой - вероятности переходов
        logits = Dense(self.num_cities)(x)
        outputs = tf.keras.activations.softmax(logits)
        
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=Adam(0.001), loss=self._custom_loss)
        return model
    
    def _custom_loss(self, y_true, y_pred):
        # y_true: маска посещенных городов (batch, cities, cities)
        # y_pred: вероятности переходов (batch, cities, cities)
        
        # Применяем маску к предсказаниям
        masked_pred = y_pred * y_true
        
        # Нормализуем вероятности
        masked_pred = masked_pred / (K.sum(masked_pred, axis=-1, keepdims=True) + K.epsilon())
        
        # Вычисляем кросс-энтропию
        loss = -K.sum(y_true * K.log(masked_pred + K.epsilon()), axis=-1)
        return K.mean(loss)
    
    def train(self, X_train, routes, epochs=50, batch_size=128):
        """
        X_train: матрицы расстояний (samples, cities, cities)
        routes: оптимальные маршруты (samples, cities)
        """
        # Создаем маски переходов для обучения
        y_masks = np.zeros_like(X_train)
        
        for i, route in enumerate(routes):
            for j in range(len(route)-1):
                from_city = route[j]
                to_city = route[j+1]
                y_masks[i, from_city, to_city] = 1
        
        self.model.fit(
            X_train,
            y_masks,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.1
        )

In [4]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


### Подготовка исходных данных и поиск точного решения методом динамического программирования занимает час

In [14]:
X = np.load('X_20x20.npy')
Y = np.load('Y_20x20.npy')
border = 60000
X_train = X[:border]
Y_train = Y[:border]
X_test = X[border:]
Y_test = Y[border:]

In [9]:
N = X_train.shape[1]
nlen = N*N
cnt = X_train.shape[0]
rand = np.random.RandomState(1)
N, cnt

(20, 60000)

### Инициализируем и обучаем модель

In [10]:
solver = TSPSolver(num_cities=N)
solver.train(X_train, Y_train, epochs=40)

I0000 00:00:1744734423.352401    1397 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22335 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6
I0000 00:00:1744734423.353748    1397 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22335 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:06:00.0, compute capability: 8.6


Epoch 1/40


I0000 00:00:1744734425.936553    1561 service.cc:152] XLA service 0x7846b000f050 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744734425.936573    1561 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
I0000 00:00:1744734425.936575    1561 service.cc:160]   StreamExecutor device (1): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2025-04-15 16:27:05.976456: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1744734426.182725    1561 cuda_dnn.cc:529] Loaded cuDNN version 90800


 75/422 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.7150e-06

I0000 00:00:1744734427.526380    1561 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


422/422 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 1.8191e-06 - val_loss: 1.1097e-06
Epoch 2/40
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.0735e-06 - val_loss: 9.9877e-07
Epoch 3/40
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 9.7349e-07 - val_loss: 8.8824e-07
Epoch 4/40
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 8.5451e-07 - val_loss: 7.5584e-07
Epoch 5/40
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 7.2371e-07 - val_loss: 6.4797e-07
Epoch 6/40
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6.2527e-07 - val_loss: 5.7812e-07
Epoch 7/40
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5.6129e-07 - val_loss: 5.3142e-07
Epoch 8/40
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5.2239e-07 - val_loss: 5.0671e-07
Epoch 9/40
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.9728e-07 - val_loss: 4.8818e-07
Epoch 10/40
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.8190e-07 - val_loss: 4.7863e-07
Epoch 11/40
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.7042e-0

### Проверка обученной модели

In [11]:
def predict_route(solver, dist_matrix, is_rnd = False, num_iter = 500):
    ap = solver.model.predict(dist_matrix[np.newaxis, ...], verbose=0)[0]
    num_cities = dist_matrix.shape[0]
    best_route = []
    best_dist = np.inf
    for _ in range(num_iter):
        current = 0
        route = [current]
        total_dist = 0
        for _ in range(num_cities-1):
            probs = ap[current].copy()
            if is_rnd: probs = np.full_like(probs, 1) 
            # Маскируем посещенные города
            probs[route] = 0
            # Выбираем следующий город
            next_city = rand.choice(range(num_cities), p=(probs / np.sum(probs)))
            route.append(next_city)
            total_dist += dist_matrix[current, next_city]
            current = next_city
        total_dist += dist_matrix[route[-1], route[0]]
        if total_dist < best_dist:
            best_dist = total_dist
            best_route = route
    return np.array(best_route), best_dist

In [12]:
def predict_geedy_route(dist_matrix):
    num_cities = dist_matrix.shape[0]
    current = 0
    route = [current]
    next_city = np.argmax(dist_matrix[current])
    route.append(next_city)
    total_dist = dist_matrix[current, next_city]
    current = next_city
    for _ in range(num_cities-2):
        a = dist_matrix[current].copy()
        # Маскируем посещенные города
        a[route] = np.inf
        # Выбираем следующий город
        next_city = np.argmin(a)
        route.append(next_city)
        total_dist += dist_matrix[current, next_city]
        current = next_city
    total_dist += dist_matrix[route[-1], route[0]]
    return np.array(route), total_dist

In [79]:
def predict_beam_search(solver, distance_matrix, beam_width=3):
    ap = solver.model.predict(distance_matrix[np.newaxis, ...], verbose=0)[0]
    num_cities = distance_matrix.shape[0]
    beams = [([0], set(range(0, num_cities)) - {0}, 0)]
    
    for i in range(1, num_cities):
        new_beams = []
        for route, remaining, dist in beams:
            # Топ-K городов 
            top_cities = sorted(remaining, key=lambda x: ap[route[-1],x], reverse=True)[:beam_width]
            for city in top_cities:
                new_route = route + [city]
                if next((x for x in new_beams if x[0] == new_route), False):
                    print(new_beams)
                    print(route)
                    print('-'*20)
                    continue
                new_remaining = remaining - {city}
                new_dist = dist + distance_matrix[new_route[-2], new_route[-1]]
                if i == num_cities-1:
                    new_dist += distance_matrix[new_route[-1], new_route[0]]
                new_beams.append((new_route, new_remaining, new_dist))
        # Выбираем лучшие beam_width вариантов
        beams = sorted(new_beams, key=lambda x: x[2])[:beam_width]
        
    # Лучший маршрут
    best_route, _, best_dist = min(beams, key=lambda x: x[2])
    return best_route, best_dist

In [101]:
ld = []
lp = []
lg = []
lr = []
lb = []
lls = []
lsa = []
for i in tqdm(range(X_test.shape[0])):
    a = X_test[i]
    #route = Y_test[i]
    #distance = sum(a[route[j],route[j+1]] for j in range(N-1))+a[route[-1],route[0]]
    #_, total_dist = predict_route(solver, a, False, 500)
    #route, dist = predict_geedy_route(a)
    #route, rdist = predict_route(solver, a, True, 500)
    #_, bdist = predict_beam_search(solver, a, beam_width=4)
    _, ls_dist = solve_tsp_local_search(a)
    _, sa_dist = solve_tsp_simulated_annealing(a)
    #ld.append(distance)
    #lp.append(total_dist)
    #lg.append(dist)
    #lr.append(rdist)
    #lb.append(bdist)
    lls.append(ls_dist)
    lsa.append(sa_dist)
    
#Y_predict = np.array(lp)
#Y_true = np.array(ld)
#Y_greedy = np.array(lg)
#Y_rnd = np.array(lr)
#Y_beam = np.array(lb)
Y_ls = np.array(lls)
Y_sa = np.array(lsa)

100%|██████████| 3000/3000 [06:36<00:00,  7.56it/s]


In [102]:
mean_absolute_percentage_error(Y_true, Y_predict), \
mean_absolute_percentage_error(Y_true, Y_beam), \
mean_absolute_percentage_error(Y_true, Y_greedy), \
mean_absolute_percentage_error(Y_true, Y_rnd), \
mean_absolute_percentage_error(Y_true, Y_ls), \
mean_absolute_percentage_error(Y_true, Y_sa)

(0.09243095293509385,
 0.08246613383664256,
 0.2061679704944246,
 0.4061705680091632,
 0.31245418930486285,
 0.21997355559984233)

In [103]:
# Сколько случаев действительно плохого прогноза
sum(((Y_predict - Y_true) / Y_true) > 0.2), \
sum(((Y_beam - Y_true) / Y_true) > 0.2), \
sum(((Y_greedy - Y_true) / Y_true) > 0.2), \
sum(((Y_rnd - Y_true) / Y_true) > 0.2), \
sum(((Y_ls - Y_true) / Y_true) > 0.2), \
sum(((Y_sa - Y_true) / Y_true) > 0.2)

(np.int64(1),
 np.int64(8),
 np.int64(1554),
 np.int64(3000),
 np.int64(2840),
 np.int64(1819))

In [94]:
%timeit predict_beam_search(solver, a, beam_width=4)

65.6 ms ± 417 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [54]:
%timeit predict_geedy_route(a)

101 μs ± 663 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
%timeit predict_route(solver, a, False, 500)

434 ms ± 2.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%timeit solve_tsp_dynamic_programming(a)

46.6 s ± 40.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%timeit solve_tsp_branch_and_bound(a)

896 ms ± 4.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit solve_tsp_local_search(a)

6.12 ms ± 98 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
%timeit solve_tsp_simulated_annealing(a)

122 ms ± 12.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
# Не стабильно
# solve_tsp_lin_kernighan(a), solve_tsp_record_to_record(a[:5,:5])